In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import MatrixProductFunctions as mp
import os
import cmath
import math
from numba import njit
import matplotlib.animation as animation
%matplotlib inline
from ipywidgets import IntProgress, HTML
from IPython.display import display
import pickle
import matplotlib.gridspec as gridspec
from datetime import datetime
from scipy.linalg import sqrtm

In [2]:
def hermit_pol_3_comb(n, x):
    if n == 0:
        return 1 * np.exp(- x**2 / 2)
    elif n == 1:
        return 2 * x * np.exp(- x**2 / 2)

    h0 = 1
    h1 = 2 * x
    for i in range(2, n + 1):
        h = (2 * x * h1 - 2 * (i - 1) * h0)*(np.exp(- x**2 / 2))**(1/(n-1))
        h1 = h1 * (np.exp(- x**2 / 2))**(1/(n-1))
        h0, h1 = h1, h

    return h1

In [3]:
def psi_fock_3(n, x):
    # Calculate the logarithm of the factorial
    log_factorial_n = sum(np.log(np.arange(1, n + 1)))

    # Calculate the logarithm of the normalization factor
    log_norm = 0.5 * (n * np.log(2) + log_factorial_n + 0.5 * np.log(np.pi))

    # Calculate the Hermite polynomial
    hermite_value = hermit_pol_3_comb(n, x)

    # Combine the results
    result = hermite_value / np.exp(log_norm)

    return result

In [4]:
fock_states = []
for i in tqdm(range(0,250)):
    fock_states.append(psi_fock_3(i, np.linspace(-10, 10, 2**13)))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:11<00:00, 17.64it/s]


In [5]:
fock_states_mps = []
for i in tqdm(range(0,250)):
    fock_states_mps.append(mp.vec_to_mps(fock_states[i], 1e-6))

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 290.67it/s]


In [ ]:
with open('C:/Users/kaprn/Desktop/MPS_directory/TensorNetworks/Tests_of_spdc/Data_for_article/alpha_20_comparison/list_of_solutions_with_tn_incr_n1n2_1e-3.pkl', 'rb') as f:
    set_of_solutions = pickle.load(f)

In [ ]:
len(set_of_solutions)

In [ ]:
def create_dm_mpo_from_mps(mps):
    dm_mpo = []
    for i in range(len(mps)):
        tensor_combination = np.einsum("ijk, abc -> iajbkc", mps[i], np.conj(mps[i]))
        shape = tensor_combination.shape
        dm_mpo.append(tensor_combination.reshape(shape[0]*shape[1], shape[2], shape[3], shape[4]*shape[5]))
    return dm_mpo

In [ ]:
def contract_two_mps(mps1, mps2, output = "tensor"):
    mps_one = mps1.copy()
    mps_two = mps2.copy()

    begin = mps_one[0].shape[0] * mps_two[0].shape[0]
    end = mps_one[-1].shape[-1] * mps_two[-1].shape[-1]

    result = np.array(1).reshape(1,1)
    if begin <= end:
        for i in range(len(mps1)):
            result = np.einsum("ia, ijk, ajc -> kc", result, mps_one[i], mps_two[i], optimize = True)
    else:
        for i in range(len(mps1)):
            result = np.einsum("kc, ijk, ajc -> ia", result, mps_one[-i-1], mps_two[-i-1], optimize = True)
    if output == 'tensor':
        return result
    else:
        return result[0][0]

In [ ]:
def contract_mpo_and_two_mps(mps1, mps2, mpo, output = 'tensor'):
    mps_one = mps1.copy()
    mps_two = mps2.copy()
    mpo_use = mpo.copy()

    begin = mps_one[0].shape[0] * mps_two[0].shape[0] * mpo_use[0].shape[0]
    end = mps_one[-1].shape[-1] * mps_two[-1].shape[-1] * mpo_use[-1].shape[-1]

    result = np.array(1).reshape(1,1,1)
    if begin <= end:
        for i in range(len(mps1)):
            result = np.einsum("abc, ajk, bjgh, cgm -> khm", result, mps_one[i], mpo_use[i], mps_two[i], optimize = True)
    else:
        for i in range(len(mps1)):
            result = np.einsum("abc, ija, gjhb, phc -> igp", result, mps_one[-i-1], mpo_use[-i-1], mps_two[-i-1], optimize = True)
    if output == 'tensor':
        return result
    else:
        return result[0][0][0]

In [ ]:
final_density_matrices = []
for k in tqdm(range(0, len(set_of_solutions), 10)):
    environment = contract_two_mps(set_of_solutions[k][:n1], set_of_solutions[k][:n1])
    environment = environment.reshape(-1)
    density_matrix_mpo = create_dm_mpo_from_mps(set_of_solutions[k][n1:])
    final_density_matrix_mpo = density_matrix_mpo.copy()
    final_density_matrix_mpo[0] = np.einsum('i, ijkl -> jkl', environment, density_matrix_mpo[0])
    shape = final_density_matrix_mpo[0].shape
    final_density_matrix_mpo[0] = final_density_matrix_mpo[0].reshape(1, shape[0], shape[1], shape[2])
    length = 150
    dm_in_ph_basis = np.zeros((length,length), dtype = complex)
    for i in range(length):
        for j in range(length):
            second_move = contract_mpo_and_two_mps(fock_states_mps[i], fock_states_mps[j], final_density_matrix_mpo, output = 'number')
            dm_in_ph_basis[i,j] = second_move
    final_density_matrices.append(dm_in_ph_basis)
    with open('C:/Users/kaprn/Desktop/MPS_directory/TensorNetworks/Tests_of_spdc/Data_for_article/alpha_20_comparison/density_matrices_converted_to_Fb_№2.pkl', 'wb') as f:
        pickle.dump(final_density_matrices, f)